In [4]:
import pandas as pd
import json, re, os, unidecode

from gensim.models import Word2Vec

from sklearn.feature_extraction.text import CountVectorizer

from keras.models import Model
from keras.layers import Dense, Conv1D, GlobalMaxPooling1D, Embedding, Input, Lambda, Add
from keras.utils.vis_utils import plot_model
import keras.backend as K

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt

import tensorflow as tf

import numpy as np

from sklearn.neighbors import NearestNeighbors

import sys, json
sys.path.append('../freebase_api')
import GstoreConnector
import datalib
import unidecode
import tqdm

embeddings_dim = 50

#set the access username and password
username = "endpoint"
password = "123"

# start a gStoreConnector with given IP and Port
gc = GstoreConnector.GstoreConnector("freebase.gstore-pku.com", 80)

# 1. Делаем запрос на 1-hop сущности (лейблы), запоминаем путь, mid
# 2. Ищем по лейблам наличие в файлах, если есть, добавляем
# 3. Делаем запрос на контекст каждой из добавленных сущностей
# 4. Кодируем по пути и контексту
# 5. Выявляем лучшие ответы и выводим

In [13]:
mid = "http://rdf.freebase.com/ns/m.027nwk2"
sparql ="""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

                SELECT ?rel ?val  ?vallabel WHERE {

                    <"""+mid+"""> ?rel ?val .
                    ?val rdfs:label ?vallabel .
                    FILTER(lang(?vallabel) = 'en') .
                    FILTER(ISURI(?val))
                }"""

In [14]:
res = json.loads(gc.query(username, password, "freebase", sparql))

In [15]:
candidates = {}
for b in res['results']['bindings']:
    candidates[b['vallabel']['value']] = (b['rel']['value'],b['val']['value'])

In [19]:
cands = []
for i in os.listdir("/home/alex/question_answering/data/answer-entities"):
    for j in list(candidates.keys()):
        if j in i and j not in cands:
            cands.append(j)

In [20]:
cands

['Canada',
 'Person',
 'Gymnast',
 'Female',
 'French',
 'Children',
 'English Language',
 'Actor',
 'Model',
 'Celebrity',
 'Bulgaria',
 'Sofia']

In [ ]:
cand_context = {}
for a in cands:
    with open('/home/alex/question_answering/data/answer-entities/{0}.txt'.format(unidecode.unidecode(a))) as f:
        cand_context[a] = f.readlines()